In [32]:
import nltk
nltk.download('stopwords')
nltk.download('perluniprops')
nltk.download('nonbreaking_prefixes')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jfuentes/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /Users/jfuentes/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /Users/jfuentes/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!
[nltk_data] Downloading package punkt to /Users/jfuentes/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jfuentes/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [2]:
##Needed to get access to mappening.utils.database since this is under mappening.ml
import sys
sys.path.insert(0,'./../..')

from mappening.utils.database import ucla_events_collection, events_ml_collection
import re
import nltk
import string
from tqdm import tqdm
import numpy as np
from nltk.tokenize.moses import MosesTokenizer
from collections import Counter
import pandas as pd


from gensim.models import Phrases
from nltk.corpus import wordnet as wn

Opening the secret door...
Got the .env secrets...
Connected to database!
Got database collections...
[nltk_data] Downloading package perluniprops to
[nltk_data]     /Users/jfuentes/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /Users/jfuentes/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jfuentes/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/jfuentes/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Login manager set up with auth Blueprint!


In [3]:
class PreprocessText:
    def __init__(self, categorizedEvents):
        """categorizedEvents should be a list of dictionaries each corresponding to an event
            X is the tokenized preprocessed text
            Y is the corresponding categories
            phraseMl is the phrase model that can further trained and used
            phrases is a list of all the phrases identified"""
        self.X = []
        self.Y = []
        tokenizer = MosesTokenizer() #tokenizers are basically an advanced split
        for e in categorizedEvents:
            text = e[u'name'] + " " + e[u'description']
            text = tokenizer.tokenize(text)
            text = self.preprocess(text)
            self.X.append(text)
            self.Y.append(e[u'category'])
        self.phraseMl = Phrases(self.X, min_count=3) #learn ml model for phrase
        self.X = list(self.phraseMl[self.X]) #use ml model for phrases
#         self.X = list(self.phraseMl[self.X]) #get triples
        self.phrases = phrases = set([w for doc in self.X for w in doc if '_' in w])
        
    def matchNotX(self, strg, search=re.compile(r'[^!#$%&()*+,-./:;<=>?@\\^_}|{~0123456789]').search):
        """make sure word has something than punctuation"""
        return bool(search(strg)) #make sure word has something other than punctuation

    def preprocess(self, text):
        """Remove all useless words and punct, make lowercase"""
        stoplist = set('for a of the and to in . / '.split())
        stoplist = set(nltk.corpus.stopwords.words('english')) | stoplist | set(string.punctuation)
        return [word.strip(string.punctuation).lower() for word in text if word not in stoplist and self.matchNotX(word)]    
        
    def topBigrams(self, texts, n, tri=False):
        """Other method of getting phrases, currently unused because phrases can be further trained(online) and saved"""
        flatTexts = []
        for text in texts:
            for word in text:
                flatTexts.append(word)
        bigram_measures = nltk.collocations.BigramAssocMeasures()
        trigram_measures = nltk.collocations.TrigramAssocMeasures()
        topAnswers = []
        if tri:
            finder = nltk.collocations.TrigramCollocationFinder.from_words(flatTexts)
            finder.apply_freq_filter(7)
            return finder.nbest(trigram_measures.pmi, n)
        else:
            finder = nltk.collocations.BigramCollocationFinder.from_words(flatTexts)
            finder.apply_freq_filter(7)
            return finder.nbest(bigram_measures.pmi, n)

In [4]:
def gatherCategorizedEvents():
    """Return panda dataframe of events with category, description, and name"""
    allCategorizedEvents = []
    allEvents = events_ml_collection.find({}, {"category": 1, "description": 1, "name": 1, "hoster": 1, "_id": 0})
    count = 0
    for e in allEvents:
        count += 1
        e['hoster'] = e['hoster']['name']
        if 'category' in e and 'description' in e and 'name' in e:
            allCategorizedEvents.append(e)
    modernEvents = reduceCategories(allCategorizedEvents)
    print count, "total events, learning from the", len(modernEvents), "well categorized events"
    return pd.DataFrame(modernEvents)

## Event Categorization

In [5]:
def someCurrentCategories():
    """Looks at current events for the categories list, to be used if Facebook changes its events in the future"""
    allCategorizedEvents = []
    allEvents = ucla_events_collection.find({}, {"category": 1, "description": 1, "name": 1, "_id": 0})
    for e in allEvents:
        if 'category' in e and 'description' in e and 'name' in e:
            allCategorizedEvents.append(e)
    skTarget = [e['category'] for e in allCategorizedEvents]
    count = sorted(list(set(skTarget)))
    print(count)
    
curListOfCategories = [u'ART', u'CAUSE', u'COMEDY_PERFORMANCE', u'DANCE', u'DRINKS', u'FILM', u'FITNESS', u'FOOD',
                       u'GAMES', u'GARDENING', u'HEALTH', u'LITERATURE', u'MEETUP', u'MUSIC', u'NETWORKING', u'PARTY',
                       u'RELIGION', u'SHOPPING', u'SPORTS', u'THEATER', u'WELLNESS']

In [6]:
def reduceCategories(events):
    """OTHER will be discarded from the training data"""
    categoryMapping = {
        u'BOOK': u'LITERATURE',
        u'COMEDY': u'COMEDY_PERFORMANCE',
        u'CLASS': u'OTHER',
        u'DINING': u'FOOD',
        u'FAMILY': u'OTHER',
        u'FESTIVAL': u'PARTY',
        u'FOOD_TASTING': u'FOOD',
        u'FUNDRAISER': u'CAUSE',
        u'LECTURE': u'OTHER',
        u'MOVIE': u'FILM',
        u'NEIGHBORHOOD': u'OTHER',
        u'NIGHTLIFE': u'OTHER',
        u'RELIGIOUS': u'RELIGION',
        u'VOLUNTEERING': u'CAUSE',
        u'WORKSHOP': u'OTHER'
    }
    
    for e in events:
        category = e['category']
        if category in categoryMapping:
            e['category'] = categoryMapping[category]
    reducedEvents = [e for e in events if e['category'] != u'OTHER']
    return reducedEvents
    

In [7]:
X = gatherCategorizedEvents()

6936 total events, learning from the 2828 well categorized events


## ML Helper Functions

In [8]:
from sklearn.cross_validation import train_test_split

def train(classifier, X, y, trails=25):
    scores = np.zeros(trails)
    for i in tqdm(range(0, trails)):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=i)

        classifier.fit(X_train, y_train)
        scores[i] = (classifier.score(X_test, y_test))
    print "Average Accuracy over %d trials: %s" % (trails, np.mean(scores))
    classifier.fit(X, y)
    return classifier
 

/anaconda2/envs/ml/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [9]:
def predictList(vectorizer, classifier, x):
    print(x)
    x = vectorizer.transform(x)
    y_pred = classifier.predict(x)
    print(y_pred)


In [10]:
def giveProbPerCategory(vectorizer, classifier, x, threshold=.15):
    print(x)
    x = vectorizer.transform(x)
    y_pred = classifier.predict_proba(x)

    strongest_category = ''
    highest_match = 0
    above_threshold = []
    for i in range(len(classifier.classes_)):
        
        if y_pred[0][i] > highest_match:
            highest_match = y_pred[0][i]
            strongest_category = classifier.classes_[i]
            
        if y_pred[0][i] > threshold:
            above_threshold.append(classifier.classes_[i])
    
    if not above_threshold:
        return [strongest_category]
    else:
        return above_threshold

# Model Making

In [11]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
 
# create the transform
vectorizer = TfidfVectorizer()

# tokenize and build vocab
X_name_transform = vectorizer.fit_transform(X['name'])
X_details_transform = vectorizer.fit_transform(X['description'])
# print(vectorizer.vocabulary_)
# print(vectorizer.idf_)

nbModel = MultinomialNB()
nbModel = train(nbModel, X_name_transform, X['category'])

100%|██████████| 25/25 [00:00<00:00, 55.27it/s]

Average Accuracy over 25 trials: 0.5812164073550212


COMBINE NAME AND DESCRIPTION

In [28]:
from sklearn.naive_bayes import MultinomialNB
from scipy.sparse import hstack
from sklearn.feature_extraction.text import TfidfVectorizer
 
# create the transform
vectorizer = TfidfVectorizer()

# tokenize and build vocab
X_name_transform = vectorizer.fit_transform(X['name'])
X_details_transform = vectorizer.fit_transform(X['description'])
X_total_transform = hstack([X_name_transform, X_details_transform])

nbModel = MultinomialNB()
nbModel = train(nbModel, X_total_transform, X['category'])

100%|██████████| 25/25 [00:01<00:00, 21.98it/s]


Average Accuracy over 25 trials: 0.5854596888260254


In [29]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack

# create the transform
vectorizer = TfidfVectorizer()

# tokenize and build vocab
X_name_transform = vectorizer.fit_transform(X['name'])
X_details_transform = vectorizer.fit_transform(X['description'])
X_total_transform = hstack([X_name_transform, X_details_transform])

nbModel = MultinomialNB()
param = {'alpha': [.01, .04, .05, .06, .3, .9, 1.5]}
gs = GridSearchCV(nbModel, param)
gs.fit(X_total_transform, X['category'])
pd.DataFrame(gs.cv_results_).sort_values('mean_test_score', ascending=False)[:2]

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_alpha,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,split1_train_score,split2_test_score,split2_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
1,0.023214,0.005026,0.696605,0.981968,0.04,{u'alpha': 0.04},1,0.692955,0.981353,0.721868,0.984624,0.674866,0.979926,0.002720,0.000722,0.019321,0.001966
3,0.025697,0.005325,0.693069,0.974367,0.06,{u'alpha': 0.06},2,0.691903,0.974427,0.721868,0.977200,0.665241,0.971474,0.000694,0.000490,0.023082,0.002338


STOPLIST 

In [27]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack

# create the transform
vectorizer = TfidfVectorizer(stop_words='english')

# tokenize and build vocab
X_name_transform = vectorizer.fit_transform(X['name'])
X_details_transform = vectorizer.fit_transform(X['description'])
X_total_transform = hstack([X_name_transform, X_details_transform])

nbModel = MultinomialNB()
param = {'alpha': [.01, .04, .05, .06, .3, .9, 1.5]}
gs = GridSearchCV(nbModel, param)
gs.fit(X_total_transform, X['category'])
pd.DataFrame(gs.cv_results_).sort_values('mean_test_score', ascending=False)[:2]

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_alpha,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,split1_train_score,split2_test_score,split2_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
1,0.025976,0.005302,0.706860,0.985860,0.04,{u'alpha': 0.04},1,0.706625,0.986148,0.729299,0.988865,0.684492,0.982567,0.001962,0.001010,0.018253,0.002579
2,0.029144,0.005285,0.703678,0.984091,0.05,{u'alpha': 0.05},2,0.702419,0.984017,0.726115,0.987275,0.682353,0.980983,0.001082,0.000526,0.017849,0.002569


Lemming

In [34]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from nltk import WordNetLemmatizer, word_tokenize

def lemma_tokenizer(text):
    wn = WordNetLemmatizer()
    tokenizer = word_tokenize
    return [wn.lemmatize(w) for w in tokenizer(text)]

# create the transform
vectorizer = TfidfVectorizer(stop_words='english', analyzer=lemma_tokenizer)

# tokenize and build vocab
X_name_transform = vectorizer.fit_transform(X['name'])
X_details_transform = vectorizer.fit_transform(X['description'])
X_total_transform = hstack([X_name_transform, X_details_transform])

nbModel = MultinomialNB()
param = {'alpha': [.01, .04, .05, .06, .3, .9, 1.5]}
gs = GridSearchCV(nbModel, param)
gs.fit(X_total_transform, X['category'])
pd.DataFrame(gs.cv_results_).sort_values('mean_test_score', ascending=False)[:2]

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_alpha,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,split1_train_score,split2_test_score,split2_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
0,0.036891,0.006079,0.689887,0.993462,0.01,{u'alpha': 0.01},1,0.680336,0.994140,0.721868,0.994698,0.667380,0.991548,0.005828,0.000244,0.023213,0.001372
1,0.029470,0.006869,0.686704,0.983558,0.04,{u'alpha': 0.04},2,0.689800,0.982952,0.715499,0.985684,0.654545,0.982039,0.000835,0.001367,0.024927,0.001549


NGrams

In [38]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack

for ngrams in [(1,1), (1,2), (1,3), (2,2), (2,3), (3,3)]:
    # create the transform
    vectorizer = TfidfVectorizer(stop_words='english', ngram_range=ngrams)

    # tokenize and build vocab
    X_name_transform = vectorizer.fit_transform(X['name'])
    X_details_transform = vectorizer.fit_transform(X['description'])
    X_total_transform = hstack([X_name_transform, X_details_transform])

    nbModel = MultinomialNB(alpha=.04)
    print(ngrams)
    nbModel = train(nbModel, X_total_transform, X['category'])

 12%|█▏        | 3/25 [00:00<00:00, 23.11it/s]

(1, 1)


100%|██████████| 25/25 [00:00<00:00, 26.52it/s]


Average Accuracy over 25 trials: 0.7342008486562942


  0%|          | 0/25 [00:00<?, ?it/s]

...

Average Accuracy over 25 trials: 0.6511456859971712


  4%|▍         | 1/25 [00:00<00:03,  6.89it/s]

(3, 3)


100%|██████████| 25/25 [00:03<00:00,  7.08it/s]


Average Accuracy over 25 trials: 0.5723338048090523


In [41]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack

# create the transform
vectorizer = TfidfVectorizer(stop_words='english')

# tokenize and build vocab
X_name_transform = vectorizer.fit_transform(X['name'])
X_details_transform = vectorizer.fit_transform(X['description'])
X_total_transform = hstack([X_name_transform, X_details_transform])

rf = RandomForestClassifier()
param = {'n_estimators': [10, 150, 300],
          'max_depth': [30,60,90, None]}
gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)
gs.fit(X_total_transform, X['category'])
pd.DataFrame(gs.cv_results_).sort_values('mean_test_score', ascending=False)[:5]

/anaconda2/envs/ml/lib/python2.7/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
/anaconda2/envs/ml/lib/python2.7/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split3_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/anaconda2/envs/ml/lib/python2.7/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split4_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_max_depth,param_n_estimators,params,rank_test_score,split0_test_score,split0_train_score,...,split2_test_score,split2_train_score,split3_test_score,split3_train_score,split4_test_score,split4_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
11,43.664493,0.433965,0.708274,0.999293,None,300,"{u'n_estimators': 300, u'max_depth': None}",1,0.706294,0.999113,...,0.710247,0.999558,0.733570,0.999558,0.666068,0.999119,5.691373,0.131625,0.023093,0.000217
7,23.899056,0.275176,0.703324,0.998408,90,150,"{u'n_estimators': 150, u'max_depth': 90}",2,0.692308,0.998227,...,0.704947,0.998232,0.717584,0.999558,0.676840,0.998239,1.093417,0.031485,0.017164,0.000601
10,29.502144,0.272200,0.703324,0.999293,None,150,"{u'n_estimators': 150, u'max_depth': None}",2,0.690559,0.999113,...,0.706714,0.999558,0.728242,0.999558,0.667864,0.999119,2.703972,0.070401,0.021978,0.000217
8,46.596309,0.501277,0.701202,0.998319,90,300,"{u'n_estimators': 300, u'max_depth': 90}",4,0.694056,0.998227,...,0.701413,0.997790,0.722913,0.999558,0.658887,0.998679,2.945843,0.061167,0.024543,0.000763
5,32.256487,0.457183,0.691655,0.990275,60,300,"{u'n_estimators': 300, u'max_depth': 60}",5,0.685315,0.990248,...,0.704947,0.988064,0.706927,0.992494,0.653501,0.990753,4.976321,0.147348,0.020596,0.001432


In [42]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack

# create the transform
vectorizer = TfidfVectorizer(stop_words='english')

# tokenize and build vocab
X_name_transform = vectorizer.fit_transform(X['name'])
X_details_transform = vectorizer.fit_transform(X['description'])
X_total_transform = hstack([X_name_transform, X_details_transform])

gb = GradientBoostingClassifier()
param = {'n_estimators': [10, 150, 300]}
gs = GridSearchCV(gb, param, cv=5, n_jobs=-1)
gs.fit(X_total_transform, X['category'])
pd.DataFrame(gs.cv_results_).sort_values('mean_test_score', ascending=False)[:5]

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_n_estimators,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,...,split2_test_score,split2_train_score,split3_test_score,split3_train_score,split4_test_score,split4_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
1,4616.345689,0.062979,0.682461,0.999204,150,{u'n_estimators': 150},1,0.681818,0.999113,0.691228,...,0.696113,0.999558,0.696270,0.999558,0.646320,0.999119,2130.005668,0.010900,0.018658,0.000332
2,1154.908539,0.116251,0.682107,0.999293,300,{u'n_estimators': 300},2,0.687063,0.999113,0.701754,...,0.694346,0.999558,0.687389,0.999558,0.639138,0.999119,585.401769,0.018500,0.021953,0.000217
0,96.969431,0.041524,0.631188,0.839109,10,{u'n_estimators': 10},3,0.627622,0.836436,0.657895,...,0.637809,0.840407,0.653641,0.834879,0.578097,0.840159,9.001699,0.014812,0.028472,0.003118


TESTING

In [ ]:
tokenizer = vectorizer.build_tokenizer()

for w in vectorizer.vocabulary_:
    if not matchNotX(w):
        print(w)

In [ ]:
sorted(list(set(skTarget)))

In [ ]:
someCurrentCategories()